In [221]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import random
random.seed(10)

In [222]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [223]:
data = pd.read_csv("C:/Santhosh/AIML/Github/Machine-Learning/Costa Rican Household Poverty Level Prediction/data/train.csv")

In [224]:
columns_del_list = ['idhogar', 'Id','dependency','tamhog','r4t3', 'tamviv', 'r4h1', 'hogar_total','r4h2', 'r4h3', 'r4m1', 'r4m2','r4m3', 'hogar_nin', 'hogar_adul', 'hogar_mayor', 'SQBescolari','SQBage','SQBhogar_total','SQBedjefe','SQBhogar_nin','SQBovercrowding','SQBdependency','SQBmeaned','agesq']

In [225]:
def drop_columns(data):
    data.drop(columns_del_list, axis=1, inplace=True)
#  Dealing with null or na
def replce_na(data):
    data['v2a1'].fillna(0, inplace=True)
    data['v18q1'].fillna(0, inplace=True)
    data['rez_esc'].fillna(0, inplace=True)
    data['meaneduc'].fillna(data['meaneduc'].median(), inplace=True)
def get_cat_cols(data):
    cols = data.columns
    num_cols = data._get_numeric_data().columns
    cat_columns = list(set(cols) - set(num_cols))
    return cat_columns
def encode_data(data):
    data['edjefe'] = data['edjefe'].replace(['yes'], 1)
    data['edjefe'] = data['edjefe'].replace(['no'], 0)
    data['edjefa'] = data['edjefa'].replace(['yes'], 1)
    data['edjefa'] = data['edjefa'].replace(['no'], 0)

In [226]:
drop_columns(data)
replce_na(data)
get_cat_cols(data)
encode_data(data)

In [227]:
# More biased towards value '4'
print("Target 1 - ", data[data.Target==1].shape[0])
print("Target 2 - ", data[data.Target==2].shape[0])
print("Target 3 - ", data[data.Target==3].shape[0])
print("Target 4 - ", data[data.Target==4].shape[0])

Target 1 -  755
Target 2 -  1597
Target 3 -  1209
Target 4 -  5996


In [228]:
targ_1 = data[data.Target==1].sample(frac=0.8)
targ_2 = data[data.Target==2].sample(frac=0.8)
targ_3 = data[data.Target==3].sample(frac=0.8)
targ_4 = data[data.Target==4].sample(frac=0.8)
train = pd.DataFrame
train  = pd.concat([targ_1, targ_2, targ_3, targ_4])
train = train.sample(frac=1) # Shuffle data
test = data.loc[~data.index.isin(train.index)]

In [229]:
X_train = train.drop(['Target'], axis =1)
Y_train = train['Target']
X_test = test.drop(['Target'], axis =1)
Y_test = test['Target']

In [230]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(70)
knn.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=70, p=2,
                     weights='uniform')

In [231]:
print('Train score for K-70 :', knn.score(X_train, Y_train)*100)
print('Train score for K-70 :',knn.score(X_test, Y_test)*100)

Train score for K-70 : 65.17133141511901
Train score for K-70 : 64.93982208267923


In [232]:
import pickle
knnPickle = open('knnpickle_file', 'wb')
pickle.dump(knn, knnPickle)

In [233]:
submission_x = pd.read_csv('C:/Santhosh/AIML/Github/Machine-Learning/Costa Rican Household Poverty Level Prediction/data/test.csv')

In [234]:
drop_columns(submission_x)
replce_na(submission_x)
get_cat_cols(submission_x)
encode_data(submission_x)

In [235]:
loaded_model = pickle.load(open('knnpickle_file', 'rb'))
result = loaded_model.predict(submission_x)

In [236]:
sample_sub = pd.read_csv('C:\Santhosh\AIML\Github\Machine-Learning\Costa Rican Household Poverty Level Prediction\data\sample_submission.csv')

In [237]:
sample_sub.Target = result

In [238]:
sub = pd.DataFrame(columns=['Id','Target'])
sub.Id = sample_sub.Id
sub.Target = result
sub.to_csv('submission.csv', encoding='utf-8', index=False)